In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [18]:
tf.__version__

'2.9.2'

#**Part 1 - Data Preprocessing --->**

**Importing dataset -->**

In [19]:
dataset = pd.read_csv('/content/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[: , -1].values

In [20]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [21]:
print(y)

[1 0 1 ... 1 1 0]


#**Encoding Categorical Data -->**

In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[: , 2] = le.fit_transform(X[ : , 2])

In [23]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


**One Hot Encoding the "Geography" column -->**

In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder() , [1])],remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [25]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Splitting the dataset into Training and Testing set -->

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

**Feature Scaling -->**

In [27]:
from sklearn.preprocessing import StandardScaler    #for scaling the data and data after scaling should have variance ="1"
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#**Part 2 - Building the ANN**

**Initialising the ANN -->**

In [28]:
ann = tf.keras.models.Sequential()

Adding the input layer and first hidden layer

In [29]:
#ann.add(tf.keras.Input(shape=(11,)))

In [31]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))            #units = 6 is most optimised

Adding the second hidden layer 

In [32]:
ann.add(tf.keras.layers.Dense(units=6,activation ='relu'))

Adding the output layer

In [33]:
ann.add(tf.keras.layers.Dense(units=1,activation ='sigmoid'))              # units =1 becoz we require only one output 0 or 1.

#**Part 3 - Training the ANN**

Compiling the ANN

In [34]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# adam is most optimised optimizer .Iske piche Gradient descent optimiser works.

Training the ANN on the Training set

In [35]:
ann.fit(X_train , y_train , batch_size = 32 , epochs = 10)

Epoch 1/10
250/250 [==============================] - 2s 2ms/step - loss: 0.5782 - accuracy: 0.7165
Epoch 2/10
250/250 [==============================] - 0s 2ms/step - loss: 0.4655 - accuracy: 0.7960
Epoch 3/10
250/250 [==============================] - 0s 2ms/step - loss: 0.4467 - accuracy: 0.8000
Epoch 4/10
250/250 [==============================] - 0s 2ms/step - loss: 0.4380 - accuracy: 0.8039
Epoch 5/10
250/250 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.8046
Epoch 6/10
250/250 [==============================] - 0s 2ms/step - loss: 0.4275 - accuracy: 0.8086
Epoch 7/10
250/250 [==============================] - 0s 2ms/step - loss: 0.4210 - accuracy: 0.8117
Epoch 8/10
250/250 [==============================] - 0s 2ms/step - loss: 0.4085 - accuracy: 0.8206
Epoch 9/10
250/250 [==============================] - 0s 2ms/step - loss: 0.3916 - accuracy: 0.8304
Epoch 10/10
250/250 [==============================] - 0s 2ms/step - loss: 0.3769 - accuracy: 0.8439

#**Part 4 - making the predictions and evaluating the model**

Predicting the result of single observation -->


Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer ?

In [36]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))> 0.5)

1/1 [==============================] - 0s 106ms/step
[[False]]


Predicting the Test set results -->

In [37]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Making the Confusion Metric -->

In [38]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1526   69]
 [ 226  179]]


0.8525